In [2]:
'''script to create the final fine-tuning dataset for fine-tuning the model with the augmented tpcds data
each entry has input,output,id,text. attribute text is the input for the fine-tuning process.'''
import os 
import json


In [3]:
#create modified dataset including prompt for fine-tuning 
import json
file_path = "/workspace/data/finetuning/dataset/sqlize-dataset.json"

with open(file_path, 'r') as file:
    sqlize_data = json.load(file)

print(len(sqlize_data))



In [5]:
finetuned_prompt=f'''
[SYSTEM]:"You are an expert Text-to-SQL generator assistant. Your goal is to provide correct SQL queries to the given text description. Your output only contains the SQL code. No explanation or introductory sentences surrounding the SQL response is needed. You are given schema information. Here is the schema information: 
{"{schema_info}"}. Here are the 5 critical rules for the interactions you must abide: <rules> 1. Do not wrap the generated SQL code within SQL code markdown format. Also, do not include the SQL keyword in the beginning of the response. 2. If I don't tell you to find the limited set of results, limit to 100. 3. Only use table and columns from the list provided 4. When performing aliasing, make sure to refer the aliased tables as alias.column_name and not as alias_column_name. 5. For US state names, use abbreviated forms. For example, for South Dakota state, use SD.</rules> 

"Here is the user question:"[/SYSTEM]
[HUMAN]:{"{user_input}"}[/HUMAN]
'''

In [6]:
finetuned_data=[]
for queryIdx in range(len(sqlize_data)):
    user_input=finetuned_prompt.format(schema_info=sqlize_data[queryIdx]["schema"],user_input=sqlize_data[queryIdx]["nlq"])
    output=sqlize_data[queryIdx]["sql"]
    finetuned_data.append({
        "id":sqlize_data[queryIdx]["id"],
        "input":user_input,
        "output":output,
        "text": f'''\n{user_input}\n[SEP] {output}'''
    })

json_string = json.dumps(finetuned_data, indent=4)




# Write the JSON string to a file
with open('/workspace/data/finetuning/dataset/sqlize-finetuned-dataset.json', 'w') as file:
    file.write(json_string)   